In [ ]:
import pandas as pd
import numpy as np


In [ ]:
arquivo_modelos = './airplane/airplanedat.csv'

df_modelos = pd.read_csv(arquivo_modelos, header=None, names=["Modelo", "Coluna2", "Coluna3"])
df_flights = pd.read_csv('viagens/flights_sample_3m.csv')

In [ ]:
# Função para extrair o fabricante da string do modelo
def extrair_fabricante(modelo):
    modelo = str(modelo).strip().replace('"', '')
    return modelo.split()[0]

# Aplica a função à coluna "Modelo"
df_modelos["Fabricante"] = df_modelos["Modelo"].apply(extrair_fabricante)

# Visualiza os primeiros 10 resultados
df_modelos.head(10)

In [ ]:
# Tabela com percentagens
dados_percentagem = {
    "Boeing": 35,
    "Airbus": 32,
    "Embraer": 8,
    "Bombardier": 6,
    "McDonnell Douglas": 5,
    "ATR": 3,
    "De Havilland": 2,
    "Fokker": 1,
    "Tupolev": 1,
    "Ilyushin": "<1",
    "Sukhoi": "<1",
    "COMAC": "<1",
    "Antonov": "<1",
    "Yakovlev": "<1",
    "British Aerospace": "<1",
    "BAe": "<1",
    "Avro": "<1",
    "Lockheed": "<1",
    "Fairchild Dornier": "<1",
    "Saab": "<1",
    "Harbin": "<1",
    "Partenavia": "<1",
    "Pilatus": "<1",
    "Piper": "<1",
    "Cessna": "<1",
    "Beechcraft": "<1",
    "Hawker": "<1",
    "Learjet": "<1",
    "Aerospatiale": "<1",
    "Concorde": 0,
    "NAMC": 0,
    "Sikorsky": "N/A",
    "Bell": "N/A"
}

In [ ]:
df_percentagem = pd.DataFrame(list(dados_percentagem.items()), columns=["Fabricante", "Percentagem de Uso Comercial (%)"])

In [ ]:
# Merge dos datasets com base no nome do fabricante
df_final = pd.merge(df_modelos, df_percentagem, how="left", left_on="Fabricante", right_on="Fabricante")

In [ ]:
df_final = df_final.drop(columns=["Coluna2", "Coluna3"])


In [ ]:
df_final["id"] = range(1, len(df_final) + 1)


In [ ]:
df_avioes = df_final


In [ ]:
def percentagem_valida(valor):
    try:
        valor_str = str(valor).replace('%', '').strip()
        if '<' in valor_str or 'N/A' in valor_str:
            return False
        return float(valor_str) >= 1
    except:
        return False


In [ ]:
# 2. Filtrar apenas os aviões que têm percentagem numérica (descartar 'N/A', '<1', etc)
df_avioes_filtrado = df_avioes[df_avioes["Percentagem de Uso Comercial (%)"].apply(percentagem_valida)].copy()
df_avioes_filtrado.head(10)



In [ ]:
# 3. Converter a percentagem para número
df_avioes_filtrado["Peso"] = df_avioes_filtrado["Percentagem de Uso Comercial (%)"].astype(float)

# 4. Normalizar os pesos para somarem 1 (distribuição de probabilidade)
df_avioes_filtrado["Peso Normalizado"] = df_avioes_filtrado["Peso"] / df_avioes_filtrado["Peso"].sum()

# 5. Gerar amostra aleatória com base nas percentagens
avioes_amostrados = np.random.choice(
    df_avioes_filtrado["Modelo"], 
    size=len(df_flights), 
    p=df_avioes_filtrado["Peso Normalizado"]
)

# 6. Adicionar ao dataframe de voos
df_flights["Modelo Avião"] = avioes_amostrados

# 7. Salvar o resultado
df_flights.to_csv("flights_com_modelos.csv", index=False)
df_flights.head(10)